# MOL518: Bio Notebook 01

## Biopython Library

In [ ]:
!pip install biopython

### 1. Seq and SeqRecord

##### Compute an amino acid distribution using a dictionary

In [ ]:
from Bio import SeqIO

filename = 'Human_proteome.fasta'

# Create an empty dictionary.
# The keys will be the amino acid abbreviations.
# The values will be the count of each amino acid, for example:
# {'A': 1691536, 'K': 1367809, ...}

aaCounts = {}

for record in SeqIO.parse(filename, 'fasta'):        
    for aa in record.seq:                   # Look through each letter in the string "line"
        if aa in aaCounts:                  # If the amino acid is already in the dictionary
            aaCounts[aa] = aaCounts[aa] + 1 # Increment the count by 1
        else:                               # If the amino acid is not yet in the dictionary.
            aaCounts[aa] = 1                # Then this is the first one.
    
print('Found {} different amino acids.'.format(len(aaCounts)))

##### Let's look at a record object

In [ ]:
# Print a record object
print(record.id)

In [ ]:
# Print all the methods and attributes
print(dir(record))

##### Search a protein by the common name in the record description

In [ ]:
prot_search = 'GN=TP53 '
# Look to find first occurance of gene P53 in the fasta file and print the sequence
search_ctr = 0
for record in SeqIO.parse(filename, 'fasta'): 
    if prot_search.lower() in record.description.lower():
        print(record.description)
        print(record.seq)
        search_ctr += 1
print('Occurances: {}'.format(search_ctr))

### 2. Pairwise Alignment of Sequences

#### Read 2 proteom files and align the first 2 genes and print alignment percentage.

##### Read the 2 protein files and define iterators

In [ ]:
from Bio import SeqIO
from Bio import Align

human_filename = 'Human_proteome.fasta'
dros_filename = 'Drosophila_proteome.faa'

# Create 2 empty dictionaries to hold the human and drosophila sequences
# The keys will be the sequence identifiers.
# The values will be the sequence.
# {'AAF45346.2': 'MEHSDPVPYQQLSTLS...', 'AAF45368.1': 'MNILRRLDRLIAP...', ...}

human_seqs = {}
dros_seqs = {}
for record in SeqIO.parse(human_filename, 'fasta'):
    human_seqs[record.id] = record.seq
for record in SeqIO.parse(dros_filename, 'fasta'):
    dros_seqs[record.id] = record.seq

#human_iter = iter(human_seqs.items())
#dros_iter = iter(dros_seqs.items())

##### Define a function to perform alignment

In [ ]:
def do_alignment(seq_target, seq_query):
    # Perform global alignment
    aligner = Align.PairwiseAligner()
    alignments = aligner.align(seq_target, seq_query)
    
    # Get the best alignment (highest score)
    best_alignment = alignments[0]
    
    # Calculate the number of aligned residues
    aligned_residues = sum(1 for start, stop in best_alignment.aligned[0] for _ in range(start, stop))
    
    # Calculate percentage of aligned residues
    total_residues = max(len(seq_target), len(seq_query))
    percentage_aligned = (aligned_residues / total_residues) * 100
    
    # Print alignment and results
    #print("Percentage identity: {:3.2f}%".format(percentage_aligned))
    #print(best_alignment)
    return percentage_aligned, aligned_residues

# Test the alignment
pct_id = do_alignment("AAACAAA", "AAAGAAA")
print(pct_id)

##### Define itertators on the dictionaries and align

In [ ]:
human_item = next(iter(human_seqs.items()))
dros_item = next(iter(dros_seqs.items()))

pct_id = do_alignment(human_item[1], dros_item[1])
print(pct_id)

##### Loop through the dictionaries and align the first human sequence against all the drosophila sequences

In [ ]:
import time
start_time = time.time()
match_counter = 0 
progress_count = 0
for dros_item in dros_seqs.items():
    pct_id, aligned_count = do_alignment(human_item[1], dros_item[1])
    if pct_id >= 30.00 and aligned_count > 100:
        match_counter += 1
    progress_count += 1
    if (progress_count % 10000) == 0:
        print('Processed:', progress_count)
end_time = time.time()
print('Matched Count:', match_counter)
print('Execution Time: ',end_time - start_time)

##### Loop through the dictionaries and align the first human sequence against all the drosophila sequences and write then out to a new file

In [ ]:
import time
start_time = time.time()
match_counter = 0 
progress_count = 0
matched_records = []
for dros_item in dros_seqs.items():
    pct_id, aligned_count = do_alignment(human_item[1], dros_item[1])
    if pct_id >= 30.00 and aligned_count > 100:
        match_counter += 1
        matched_records.append(record)
    progress_count += 1
    if (progress_count % 10000) == 0:
        print('Processed:', progress_count)
SeqIO.write(matched_records, human_item[0].replace('|', '_') + "_dros.fasta", "fasta")
end_time = time.time()
print('Matched Count:', match_counter)
print('Execution Time: ',end_time - start_time)

##### Inspect the aligner and try different alignments

In [ ]:
print(aligner)

##### Try different scoring matrices

In [ ]:
from Bio.Align import substitution_matrices
matrix = substitution_matrices.load("BLOSUM62")